## Porting the Marching Cubes algorithm 
The following SYCL code below is a port of the implementation in the CUDA SDK.

1. Inspect the following code cell and click __Run__ ▶ to save the code to a file.
2. Next run -- the cell in the __Build and Run__ section below the code to compile and execute the code.

## Kernels

### Classify Voxel

In [ ]:
%%writefile src/kernelClassifyVoxel.cpp

void classifyVoxel(uint *voxelVerts, 
                   uint *voxelOccupied, 
                   uchar *volume,
                   sycl::uint3 gridSize, 
                   sycl::uint3 gridSizeShift,
                   sycl::uint3 gridSizeMask, 
                   uint numVoxels,
                   sycl::float3 voxelSize, 
                   float isoValue,
                   sycl::accessor<uint, 1, sycl::access_mode::read> numVertsAcc,
                   sycl::accessor<uchar, 1, sycl::access_mode::read> volumeAcc,
                   sycl::id<3> idx) {

  uint i = idx[2] * gridSize.x() * gridSize.y() + idx[1] * gridSize.x() + idx[0];

  sycl::uint3 gridPos = calcGridPos(i, gridSizeShift, gridSizeMask);

  float field[8];
  field[0] = sampleVolume(volumeAcc, volume, gridPos, gridSize);
  field[1] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(1, 0, 0), gridSize);
  field[2] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(1, 1, 0), gridSize);
  field[3] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(0, 1, 0), gridSize);
  field[4] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(0, 0, 1), gridSize);
  field[5] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(1, 0, 1), gridSize);
  field[6] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(1, 1, 1), gridSize);
  field[7] = sampleVolume(volumeAcc, volume, gridPos + sycl::uint3(0, 1, 1), gridSize);

  uint cubeindex;
  cubeindex = uint(field[0] < isoValue);
  cubeindex += uint(field[1] < isoValue) * 2;
  cubeindex += uint(field[2] < isoValue) * 4;
  cubeindex += uint(field[3] < isoValue) * 8;
  cubeindex += uint(field[4] < isoValue) * 16;
  cubeindex += uint(field[5] < isoValue) * 32;
  cubeindex += uint(field[6] < isoValue) * 64;
  cubeindex += uint(field[7] < isoValue) * 128;

  uint numVerts = numVertsAcc[cubeindex];

  if (i < numVoxels) {
    voxelVerts[i] = numVerts;
    voxelOccupied[i] = (numVerts > 0);
  }
}

### Launch classifyVoxel

In [ ]:
%%writefile src/kernelLaunchClassifyVoxel.cpp

extern "C" 
void launch_classifyVoxel(sycl::queue &q, 
                          sycl::range<3> globalRange,
                          uint *voxelVerts, 
                          uint *voxelOccupied, 
                          uchar *volume,
                          sycl::uint3 gridSize, 
                          sycl::uint3 gridSizeShift,
                          sycl::uint3 gridSizeMask, 
                          uint numVoxels,
                          sycl::float3 voxelSize, 
                          float isoValue) 
{
  q.submit([&](sycl::handler &h) 
  {
    auto out = sycl::stream(1024, 768, h); 
    auto numVertsAcc = numVertsTableBuf -> get_access<sycl::access_mode::read>(h);
    auto volumeAcc = volumeBuf -> get_access<sycl::access_mode::read>(h);
        
    h.parallel_for(globalRange, [=](sycl::id<3> idx) 
    {
      classifyVoxel(voxelVerts, voxelOccupied, volume, gridSize,
                    gridSizeShift, gridSizeMask, numVoxels, voxelSize,
                    isoValue, numVertsAcc, volumeAcc, idx);
    }); 
  }).wait();
}

### Compact voxels

In [ ]:
%%writefile src/kernelCompactVoxels.cpp

void compactVoxels(uint *compactedVoxelArray, 
                   uint *voxelOccupied,
                   uint *voxelOccupiedScan, 
                   uint numVoxels, 
                   sycl::id<3> idx) 
{
  uint i = idx[2] * numVoxels * numVoxels + idx[1] * numVoxels + idx[0];
  if (i >= numVoxels) return;
  
  if (voxelOccupied[i]) {
    compactedVoxelArray[voxelOccupiedScan[i]] = i;
  }
}

### Launch compact voxels

In [ ]:
%%writefile src/kernelLaunchCompactVoxels.cpp

extern "C" 
void launch_compactVoxels(sycl::queue &q, 
                          sycl::range<3> globalRange,
                          uint *compactedVoxelArray, 
                          uint *voxelOccupied,
                          uint *voxelOccupiedScan, 
                          uint numVoxels) 
{
  q.parallel_for(globalRange, [=](sycl::id<3> idx) 
  {
    compactVoxels(compactedVoxelArray, voxelOccupied, voxelOccupiedScan, numVoxels, idx);
  }).wait();
}

### generateTriangles kernel

In [ ]:
%%writefile src/kernelGenerateTriangles.cpp

void generateTriangles(sycl::float4 *pos, 
                       sycl::float4 *norm, 
                       uint *compactedVoxelArray,
                       uint *numVertsScanned, 
                       sycl::uint3 gridSize,
                       sycl::uint3 gridSizeShift, 
                       sycl::uint3 gridSizeMask,
                       sycl::float3 voxelSize, 
                       float isoValue,
                       uint activeVoxels, 
                       uint maxVerts,
                       sycl::accessor<uint, 1, sycl::access_mode::read> triTableAcc,
                       sycl::accessor<uint, 1, sycl::access_mode::read> numVertsAcc,
                       sycl::id<3> idx) 
{
  uint i = idx[2] * gridSize.x() * gridSize.y() + idx[1] * gridSize.x() + idx[0];

  if (i >= activeVoxels) return;

  uint voxel = compactedVoxelArray[i];

  sycl::uint3 gridPos = calcGridPos(voxel, gridSizeShift, gridSizeMask);

  sycl::float3 p;
  p.x() = -1.0f + (gridPos.x() * voxelSize.x());
  p.y() = -1.0f + (gridPos.y() * voxelSize.y());
  p.z() = -1.0f + (gridPos.z() * voxelSize.z());

  sycl::float3 v[8];
  v[0] = p;
  v[1] = p + sycl::float3(voxelSize.x(), 0, 0);
  v[2] = p + sycl::float3(voxelSize.x(), voxelSize.y(), 0);
  v[3] = p + sycl::float3(0, voxelSize.y(), 0);
  v[4] = p + sycl::float3(0, 0, voxelSize.z());
  v[5] = p + sycl::float3(voxelSize.x(), 0, voxelSize.z());
  v[6] = p + sycl::float3(voxelSize.x(), voxelSize.y(), voxelSize.z());
  v[7] = p + sycl::float3(0, voxelSize.y(), voxelSize.z());

  sycl::float4 field[8];
  field[0] = fieldFunc4(v[0]);
  field[1] = fieldFunc4(v[1]);
  field[2] = fieldFunc4(v[2]);
  field[3] = fieldFunc4(v[3]);
  field[4] = fieldFunc4(v[4]);
  field[5] = fieldFunc4(v[5]);
  field[6] = fieldFunc4(v[6]);
  field[7] = fieldFunc4(v[7]);

  uint cubeindex;
  cubeindex = uint(field[0].w() < isoValue);
  cubeindex += uint(field[1].w() < isoValue) * 2;
  cubeindex += uint(field[2].w() < isoValue) * 4;
  cubeindex += uint(field[3].w() < isoValue) * 8;
  cubeindex += uint(field[4].w() < isoValue) * 16;
  cubeindex += uint(field[5].w() < isoValue) * 32;
  cubeindex += uint(field[6].w() < isoValue) * 64;
  cubeindex += uint(field[7].w() < isoValue) * 128;

  sycl::float3 vertlist[12];
  sycl::float3 normlist[12];

  vertexInterp2(isoValue, v[0], v[1], field[0], field[1], vertlist[0], normlist[0]);
  vertexInterp2(isoValue, v[1], v[2], field[1], field[2], vertlist[1], normlist[1]);
  vertexInterp2(isoValue, v[2], v[3], field[2], field[3], vertlist[2], normlist[2]);
  vertexInterp2(isoValue, v[3], v[0], field[3], field[0], vertlist[3], normlist[3]);
  vertexInterp2(isoValue, v[4], v[5], field[4], field[5], vertlist[4], normlist[4]);
  vertexInterp2(isoValue, v[5], v[6], field[5], field[6], vertlist[5], normlist[5]);
  vertexInterp2(isoValue, v[6], v[7], field[6], field[7], vertlist[6], normlist[6]);
  vertexInterp2(isoValue, v[7], v[4], field[7], field[4], vertlist[7], normlist[7]);
  vertexInterp2(isoValue, v[0], v[4], field[0], field[4], vertlist[8], normlist[8]);
  vertexInterp2(isoValue, v[1], v[5], field[1], field[5], vertlist[9], normlist[9]);
  vertexInterp2(isoValue, v[2], v[6], field[2], field[6], vertlist[10], normlist[10]);
  vertexInterp2(isoValue, v[3], v[7], field[3], field[7], vertlist[11], normlist[11]);

  uint numVerts = numVertsAcc[cubeindex];

  for (int i = 0; i < numVerts; i++) {
    uint edge = triTableAcc[cubeindex * 16 + i];

    uint index = numVertsScanned[voxel] + i;

    if (index < maxVerts) {
      pos[index] = sycl::float4{vertlist[edge].x(), vertlist[edge].y(), vertlist[edge].z(), 1.0f};
      norm[index] = sycl::float4{normlist[edge].x(), normlist[edge].y(), normlist[edge].z(), 0.0f};
    }
  }
}


### Launch generateTriangles kernel

In [ ]:
%%writefile src/kernelLaunchGenerateTriangles.cpp

extern "C" 
void launch_generateTriangles(sycl::queue &q, sycl::range<3> globalRange,
                              sycl::float4 *pos, 
                              sycl::float4 *norm,
                              uint *compactedVoxelArray, 
                              uint *numVertsScanned,
                              sycl::uint3 gridSize, 
                              sycl::uint3 gridSizeShift,
                              sycl::uint3 gridSizeMask, 
                              sycl::float3 voxelSize,
                              float isoValue, 
                              uint activeVoxels, 
                              uint maxVerts) 
{
  q.submit([&](sycl::handler &h) 
  {
    auto triTableAcc = triTableBuf->get_access<sycl::access_mode::read>(h);
    auto numVertsAcc = numVertsTableBuf->get_access<sycl::access_mode::read>(h);

    h.parallel_for(globalRange, [=](sycl::id<3> idx) 
    {
      generateTriangles(pos, norm, compactedVoxelArray, numVertsScanned,
                            gridSize, gridSizeShift, gridSizeMask, voxelSize,
                            isoValue, activeVoxels, maxVerts, triTableAcc,
                            numVertsAcc, idx);
    });
  }).wait();
}

#### Build and Run
Select the cell below and click __Run__ ▶ to compile the code:

In [ ]:
! cd src ; icpx -fsycl marchingCubes.cpp marchingCubesKernels.cpp -o ../build/mc


Select the cell below and click __Run__ ▶ to execute the code on selected device:

In [ ]:
! pwd

In [ ]:
! ./build/mc